In [ ]:
import urllib.request
import numpy as np
import pandas as pd
import pickle
import utils
import further_embeddings as data # takes a while because it loads all embedding datasets

successfully loaded utils


In [ ]:
# load the two debiased embeddings obtained by our implementation of Double-Hard Debias
file_1 = open('debiased_1/debiased_1.p', 'wb')
debiased_equal = pickle.load(file_1)
file_1.close()
file_2 = open('debiased_2/debiased_2.p', 'wb')
debiased_fem_male = pickle.load(file_2)
file_2.close()
# load the Double-Hard debiased embeddings obtained by Wang et al. (2020)
file_3 = open('glove_dhd.p', 'wb')
glove_dhd_wang = pickle.load(file_3)
file_3.close()

In [ ]:
dict_embeddings = {'original_glove': (embedding, vocab, w2id), 'glove_dhd_wang': (glove_dhd_wang, vocab, w2id), 
                   'debiased_equal': (debiased_equal, vocab, w2id), 'debiased_fem_male': (debiased_fem_male, vocab, w2id)}

## Word Embedding Association Test
Implementation taken from https://github.com/shivaomrani/HumanBiasInSemantics with minor adjustments such as variable names for readability. See file `weat.py`.

In [2]:
from weat import weat
# auf embedding zugreifen, wenn wir das als Datei haben

In [3]:
# Career and family
# Change from Bill to Tom as in paper to avoid ambiguity
male_names = ["john", "paul", "mike", "kevin", "steve", "greg", "jeff", "tom"]
female_names = ["amy", "joan", "lisa", "sarah", "diana", "kate", "ann", "donna"]
career_attributes = ["executive", "management", "professional", "corporation", "salary", "office", "business", "career"]
family_attributes = ["home", "parents", "children", "family", "cousins", "marriage", "wedding", "relatives"]

In [4]:
# Math and arts
math_words = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
arts_words1 = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
male_attributes1 = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female_attributes1 = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]

In [5]:
# Science and arts
science_words = ["science", "technology", "physics", "chemistry", "einstein", "nasa", "experiment", "astronomy"]
arts_words2 = ["poetry", "art", "shakespeare", "dance", "literature", "novel", "symphony", "drama"]
male_attributes2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_attributes2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "her"]

In [6]:
iterations = 100000
wea_test = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding, data.w2id)
pvalue, effect_size, sd = wea_test.getPValueAndEffect()
print("p-value: ", pvalue)
print("effect size: ", effect_size)
print("standard deviation: ", sd)

The difference of means is  0.144274442398455
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
p-value:  0.0001515339304264396
effect size:  1.8059961891361644
standard deviation:  0.03986284483860918


In [7]:
glove = []
wea_test_glove_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding, data.w2id)
pvalue, effect_size, sd = wea_test_glove_1.getPValueAndEffect()
glove.append(effect_size)
glove.append(pvalue)
wea_test_glove_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding, data.w2id)
pvalue, effect_size, sd = wea_test_glove_2.getPValueAndEffect()
glove.append(effect_size)
glove.append(pvalue)
wea_test_glove_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding, data.w2id)
pvalue, effect_size, sd = wea_test_glove_3.getPValueAndEffect()
glove.append(effect_size)
glove.append(pvalue)

The difference of means is  0.144274442398455
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.01738401491456898
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.031252209897502325
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [8]:
gn = []
wea_test_gn_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_gn, data.w2id_gn)
pvalue, effect_size, sd = wea_test_gn_1.getPValueAndEffect()
gn.append(effect_size)
gn.append(pvalue)
wea_test_gn_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_gn, data.w2id_gn)
pvalue, effect_size, sd = wea_test_gn_2.getPValueAndEffect()
gn.append(effect_size)
gn.append(pvalue)
wea_test_gn_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_gn, data.w2id_gn)
pvalue, effect_size, sd = wea_test_gn_3.getPValueAndEffect()
gn.append(effect_size)
gn.append(pvalue)

The difference of means is  0.15499514529074077
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.007112938932550605
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.034820708526240196
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [9]:
gn_a = []
wea_test_gn_a_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_gn_a, data.w2id_gn_a)
pvalue, effect_size, sd = wea_test_gn_a_1.getPValueAndEffect()
gn_a.append(effect_size)
gn_a.append(pvalue)
wea_test_gn_a_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_gn_a, data.w2id_gn_a)
pvalue, effect_size, sd = wea_test_gn_a_2.getPValueAndEffect()
gn_a.append(effect_size)
gn_a.append(pvalue)
wea_test_gn_a_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_gn_a, data.w2id_gn_a)
pvalue, effect_size, sd = wea_test_gn_a_3.getPValueAndEffect()
gn_a.append(effect_size)
gn_a.append(pvalue)

The difference of means is  0.12253738998697372
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.01958712745545199
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.03415733441215707
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [10]:
hd = []
wea_test_hd_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_hd, data.w2id_hd)
pvalue, effect_size, sd = wea_test_hd_1.getPValueAndEffect()
hd.append(effect_size)
hd.append(pvalue)
wea_test_hd_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_hd, data.w2id_hd)
pvalue, effect_size, sd = wea_test_hd_2.getPValueAndEffect()
hd.append(effect_size)
hd.append(pvalue)
wea_test_hd_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_hd, data.w2id_hd)
pvalue, effect_size, sd = wea_test_hd_3.getPValueAndEffect()
hd.append(effect_size)
hd.append(pvalue)

The difference of means is  0.07614666296285577
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.014647434276412241
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.005444272304885089
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [11]:
hd_a = []
wea_test_hd_a_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_hd_a, data.w2id_hd_a)
pvalue, effect_size, sd = wea_test_hd_a_1.getPValueAndEffect()
hd_a.append(effect_size)
hd_a.append(pvalue)
wea_test_hd_a_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_hd_a, data.w2id_hd_a)
pvalue, effect_size, sd = wea_test_hd_a_2.getPValueAndEffect()
hd_a.append(effect_size)
hd_a.append(pvalue)
wea_test_hd_a_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_hd_a, data.w2id_hd_a)
pvalue, effect_size, sd = wea_test_hd_a_3.getPValueAndEffect()
hd_a.append(effect_size)
hd_a.append(pvalue)

The difference of means is  0.07614665298024192
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.01504139811731875
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.005642128220642917
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [12]:
gp = []
wea_test_gp_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_gp, data.w2id_gp)
pvalue, effect_size, sd = wea_test_gp_1.getPValueAndEffect()
gp.append(effect_size)
gp.append(pvalue)
wea_test_gp_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_gp, data.w2id_gp)
pvalue, effect_size, sd = wea_test_gp_2.getPValueAndEffect()
gp.append(effect_size)
gp.append(pvalue)
wea_test_gp_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_gp, data.w2id_gp)
pvalue, effect_size, sd = wea_test_gp_3.getPValueAndEffect()
gp.append(effect_size)
gp.append(pvalue)

The difference of means is  0.1317770363530144
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.031406250345753506
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.029088765440974385
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [13]:
gp_gn = []
wea_test_gp_gn_1 = weat(male_names, female_names, career_attributes, family_attributes, iterations, data.embedding_gp_gn, data.w2id_gp_gn)
pvalue, effect_size, sd = wea_test_gp_gn_1.getPValueAndEffect()
gp_gn.append(effect_size)
gp_gn.append(pvalue)
wea_test_gp_gn_2 = weat(math_words, arts_words1, male_attributes1, female_attributes1, iterations, data.embedding_gp_gn, data.w2id_gp_gn)
pvalue, effect_size, sd = wea_test_gp_gn_2.getPValueAndEffect()
gp_gn.append(effect_size)
gp_gn.append(pvalue)
wea_test_gp_gn_3 = weat(science_words, arts_words2, male_attributes2, female_attributes2, iterations, data.embedding_gp_gn, data.w2id_gp_gn)
pvalue, effect_size, sd = wea_test_gp_gn_3.getPValueAndEffect()
gp_gn.append(effect_size)
gp_gn.append(pvalue)

The difference of means is  0.14423039597750176
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  -0.0004306499467929825
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
The difference of means is  0.02560682910552714
Generating null distribution...
Number of permutations  100000
Getting the entire distribution


In [15]:
test_results = []
test_results.append(glove)
test_results.append(gn)
test_results.append(gn_a)
test_results.append(gp)
test_results.append(gp_gn)
test_results.append(hd)
test_results.append(hd_a)
test_results = np.array(test_results).astype(float)
row_names = ["GloVe", "GN-GloVe", "GN-GloVe(a)", "GP-GloVe", "GP-GN-GloVe", "Hard-GloVe", "Strong-Hard-GloVe"]
column_names = ["C & F: d", "C & F: p", "M & A: d", "M & A: p", "S & A: d", "S & A: p"]
df = pd.DataFrame(data = test_results, index = row_names, columns = column_names)
print(df)

                   C & F: d  C & F: p  M & A: d  M & A: p  S & A: d  S & A: p
GloVe              1.805996  0.000144  0.688595  0.084699  1.129869  0.011878
GN-GloVe           1.821105  0.000137 -0.256358  0.695582  1.068954  0.016364
GN-GloVe(a)        1.755476  0.000221  0.503003  0.159387  0.879733  0.039781
GP-GloVe           1.805885  0.000155  1.208529  0.007637  1.106440  0.013436
GP-GN-GloVe        1.797431  0.000156 -0.012642  0.511153  0.846007  0.045537
Hard-GloVe         1.546646  0.000978 -0.982604  0.975189 -0.538376  0.858503
Strong-Hard-GloVe  1.546646  0.000989 -0.985681  0.975160 -0.547118  0.862593


## Word Analogy
The word analogy task is to find a word *D* such that "*A* is to *B* as *C* is to *D*". Wang et al. (2020) have evaluated all non-debiased and debiased embeddings on the MSR word analogy task [Mikolov et al., 2013a](https://www.aclweb.org/anthology/N13-1090/) as well as on a second Google word analogy dataset [Mikolov et al. 2013b](https://arxiv.org/abs/1301.3781v3). The evaluation metric is the percentage of questions for which the correct answer is assigned the maximum score by the algorithm. The analogy task is used to show whether a debiasing method is capable of preserving desired distance relations between words. The implementation was taken from Wang et al. (2020) with some adjustments to get it running: `analogy_tasks.py`

In [16]:
import analogy_tasks as ana

### MSR

In [19]:
# MSR on all embeddings:
for key, (embedding, vocab, w2id) in dict_embeddings.items():
    print(key, ": ")
    ana.evaluate_analogy_msr(embedding, vocab, w2id)

4884
ACCURACY TOP1-MSR: 54.40% (2657/4884)


In [20]:
# ana.evaluate_analogy_msr(debiased_1, data.vocab, data.w2id)

In [21]:
ana.evaluate_analogy_google(data.embedding, data.vocab, data.w2id)

capital-common-countries.txt:
ACCURACY TOP1: 98.95% (283/286)
capital-world.txt:
ACCURACY TOP1: 94.69% (1409/1488)
currency.txt:
ACCURACY TOP1: 7.63% (18/236)
city-in-state.txt:
ACCURACY TOP1: 77.49% (1855/2394)
family.txt:
ACCURACY TOP1: 71.67% (301/420)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 9.25% (86/930)
gram2-opposite.txt:
ACCURACY TOP1: 29.22% (135/462)
gram3-comparative.txt:
ACCURACY TOP1: 78.68% (1048/1332)
gram4-superlative.txt:
ACCURACY TOP1: 46.55% (378/812)
gram5-present-participle.txt:
ACCURACY TOP1: 47.42% (441/930)
gram6-nationality-adjective.txt:
ACCURACY TOP1: 93.23% (1418/1521)
gram7-past-tense.txt:
ACCURACY TOP1: 34.87% (544/1560)
gram8-plural.txt:
ACCURACY TOP1: 75.46% (898/1190)
gram9-plural-verbs.txt:
ACCURACY TOP1: 45.57% (370/812)
Questions seen/total: 73.54% (14373/19544)
Semantic accuracy: 80.14%  (3866/4824)
Syntactic accuracy: 55.69%  (5318/9549)
Total accuracy: 63.90%  (9184/14373)


In [22]:
# ana.evaluate_analogy_google(debiased_2, data.vocab, data.w2id)